In [ ]:
#impor packages
import pandas as pd
import numpy as np
import os
import pickle
import json
import re
pd.set_option('display.max_columns', None)

In [ ]:
# location of raw data files
input_path = "/home/jupyter/datasets/raw/"

In [ ]:
#read PROCEDUREEVENTS_MV.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'PROCEDUREEVENTS_MV.csv',dtype = "object")

In [ ]:
df.info(verbose=True, null_counts=True)

In [ ]:
df.head()

In [ ]:
df = df.drop(labels = ['STARTTIME','ENDTIME','CGID','ORDERID','LINKORDERID',
                  'SECONDARYORDERCATEGORYNAME','ISOPENBAG','CONTINUEINNEXTDEPT',
                  'CANCELREASON','COMMENTS_EDITEDBY','COMMENTS_CANCELEDBY','COMMENTS_DATE'], axis = 'columns')

In [ ]:
df.head()

In [ ]:
#read PROCEDUREEVENTS_MV.csv file
df1 = pd.read_csv(filepath_or_buffer=input_path+'D_ITEMS.csv',dtype = "object")

In [ ]:
df1.info(verbose=True, null_counts=True)

In [ ]:
df1.head()

In [ ]:
df = df.merge(df1[['ITEMID','LABEL']], how='left',left_on='ITEMID',right_on='ITEMID')

In [ ]:
df = df.fillna('NaN')

In [ ]:
df.head()

In [ ]:
# Merge ITEMID with respective VALUE and UOM
df['VALUE_UOM'] = df['ITEMID']+'-'+df['VALUE']+'-'+df['VALUEUOM']

# if VALUE is missing map VALEU_UOM as 'NaN'
df.loc[df['VALUE']=='NaN','VALUE_UOM'] = 'NaN' 

df = df.drop(labels = ['VALUE','VALUEUOM', 'ITEMID'], axis = 'columns' )

In [ ]:
# Remove special charecters from the free text columns
df['LABEL'] = df['LABEL'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['LOCATION'] = df['LOCATION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['LOCATIONCATEGORY'] = df['LOCATIONCATEGORY'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['ORDERCATEGORYNAME'] = df['ORDERCATEGORYNAME'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['ORDERCATEGORYDESCRIPTION'] = df['ORDERCATEGORYDESCRIPTION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

df['STATUSDESCRIPTION'] = df['STATUSDESCRIPTION'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

In [ ]:
# create procedure events
df['procedureevents'] = df['LABEL']+' '+df['VALUE_UOM']+' '+df['ORDERCATEGORYNAME']+' '+df['ORDERCATEGORYDESCRIPTION']+' '+df['STATUSDESCRIPTION']+' '+df['LOCATION']+' '+df['LOCATIONCATEGORY']

In [ ]:
# drop redundant columns
df = df.drop(labels=['LABEL','VALUE_UOM','LOCATION','LOCATIONCATEGORY',
                     'ORDERCATEGORYNAME','ORDERCATEGORYDESCRIPTION',
                     'STATUSDESCRIPTION'], axis = 'columns')

In [ ]:
df.head()

In [ ]:
df['procedureevents'] = df['procedureevents'].map(lambda x: x.split())

In [ ]:
# remove 'NaN' form all events
df['procedureevents'] = df['procedureevents'].map(lambda x: list(filter(lambda a: a!='NaN',x)))

In [ ]:
df.head()

In [ ]:
df = df.drop(labels=['ROW_ID'], axis = 'columns')

In [ ]:
# create output path
mypath_output = "/home/jupyter/datasets/procedureevents/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [ ]:
# export the dataframe to JSON format
df.to_json(mypath_output+"procedureevents.json",orient = 'records') 